In [ ]:
import os
import io
import sys
from dotenv import load_dotenv
from google import genai
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [ ]:
load_dotenv(override=True)

In [ ]:
GEMINI_API_KEY = os.getenv(key="GEMINI_API_KEY")

if GEMINI_API_KEY:
    print("GEMINI_API_KEY found.")
else:
    raise "API Key not found."

gemini = genai.Client(api_key=GEMINI_API_KEY)

In [ ]:
GEMINI_2_0_MODEL = "gemini-2.0-flash"

ACTIVE_MODEL = GEMINI_2_0_MODEL

In [ ]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [ ]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [ ]:
def messages_for(python):
    return [{"role": "user", "parts": [{"text": user_prompt_for(python)}]}]

In [ ]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("code_converter/cpp/optimized.cpp", "w") as f:
        f.write(code)

In [ ]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_sub_array_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_sub_array_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_sub_array_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_sub_array_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Sub-array Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
def stream_gemini(python):
    stream = gemini.models.generate_content_stream(
      model=ACTIVE_MODEL,
      contents= messages_for(python),
      config={"system_instruction": {"text": system_message}}
    )

    reply = ""
    for chunk in stream:
        fragment = chunk.candidates[0].content.parts[0].text or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
def optimize(python, model):
    if model == "Gemini 2.0 flash":
        result = stream_gemini(python)
    else:
        raise ValueError("Unknown model")

    for stream_so_far in result:
        yield stream_so_far

In [ ]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [ ]:
def execute_cpp(code):
    write_output(code)
    try:
        compile_cmd = [
            "clang++",
            "-Ofast",
            "-std=c++17",
            "-march=armv8.5-a",
            "-mtune=apple-m1",
            "-mcpu=apple-m1",
            "-o",
            "code_converter/cpp/optimized",
            "code_converter/cpp/optimized.cpp",
        ]
        compile_result = subprocess.run(
            compile_cmd, check=True, text=True, capture_output=True
        )
        run_cmd = ["./code_converter/cpp/optimized"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [ ]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(
            ["Gemini 2.0 flash"],
            label="Select model",
            value="Gemini 2.0 flash",
        )
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)